# MNSIT RNN test

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
import timeit
from torchnet import meter
%matplotlib inline

import torch.nn.functional as F

# 载入数据

In [4]:
set_train = dset.MNIST('../MNIST', train=True, transform=T.ToTensor(), download=True)
loader_train = DataLoader(set_train, batch_size=512)
set_test = dset.MNIST('../MNIST', train=False, transform=T.ToTensor(),download=True)
loader_test = DataLoader(set_test, batch_size=512)

### 看下大小

In [5]:
print("训练集大小：",set_train.train_data.size())
print("训练集标签：",set_train.train_labels.size())
print("测试集大小：",set_test.test_data.size())
print("测试集标签：",set_test.test_labels.size())
type(set_train[0]) #数据集的索引是tuple

训练集大小： torch.Size([60000, 28, 28])
训练集标签： torch.Size([60000])
测试集大小： torch.Size([10000, 28, 28])
测试集标签： torch.Size([10000])


tuple

# 初始化一些参数

In [6]:
# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()
        
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # 读取 N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
    
def train(model, loss_fn, optimizer, num_epochs = 5, print_every = 200):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        check_accuracy(model, loader_test)
        model.train()
        loss_meter = meter.AverageValueMeter()
        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.cuda())
            x_var = x_var.squeeze()
            #print(x_var.shape)
            y_var = Variable(y.cuda().long())
            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            loss_meter.add(loss.data[0])
                   
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (t + 1) % print_every == 0:
                print("t:{}:loss:{:.3}".format(t,loss_meter.value()[0]))

def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    with torch.no_grad(): 
        for x, y in loader:
            x_var = Variable(x.cuda())
            x_var = x_var.squeeze()
            scores = model(x_var)
            _, preds = scores.data.cpu().max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

# 构造模型 MyLeNet

In [ ]:
class Net(nn.Module):
    def __init__(self, h_dim = 64, num_layer=2):
        super(Net, self).__init__()
        self.h_dim=h_dim
        self.num_layer = num_layer
        self.gru = nn.GRU(28,h_dim, num_layer, batch_first=True, dropout=0.2, bidirectional=False)
        self.fc1 = nn.Linear(h_dim,1000)
        self.relu= nn.ReLU()
        self.fc2 = nn.Linear(1000,10)
    def forward(self, x):
        n = x.size(0)
        h0 = x.new(self.num_layer,n,self.h_dim).fill_(0)
        out,_ = self.gru(x,h0)
        out = out[:,-1,:]
        out = out.contiguous().view(n,-1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    

net =Net(256,3).cuda()

x = torch.randn(64, 28, 28).cuda()
x_var = Variable(x) 

net(x_var).shape       

torch.Size([64, 10])

# 训练

In [ ]:
loss_fn = nn.CrossEntropyLoss().cuda()
#optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)

#torch.cuda.random.manual_seed(123)
#Net.apply(reset) #重置权值
train(net, loss_fn, optimizer, num_epochs=20)
check_accuracy(net, loader_test)

Starting epoch 1 / 20
Checking accuracy on test set
Got 977 / 10000 correct (9.77)
Starting epoch 2 / 20
Checking accuracy on test set
Got 1304 / 10000 correct (13.04)
Starting epoch 3 / 20
Checking accuracy on test set
Got 1135 / 10000 correct (11.35)
Starting epoch 4 / 20
Checking accuracy on test set
Got 1135 / 10000 correct (11.35)
Starting epoch 5 / 20
Checking accuracy on test set
Got 1135 / 10000 correct (11.35)
Starting epoch 6 / 20
Checking accuracy on test set
Got 1135 / 10000 correct (11.35)
Starting epoch 7 / 20
Checking accuracy on test set
Got 1135 / 10000 correct (11.35)
Starting epoch 8 / 20
Checking accuracy on test set
Got 1135 / 10000 correct (11.35)
